<a href="https://colab.research.google.com/github/pratjz/Tensorflow/blob/master/5_Transfer_Learning_and_Fine_Tunning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![alt text](https://live.staticflickr.com/4544/38228876666_3782386ca7_b.jpg)

## Stage 1: Install dependencies and setting up GPU environment

In [1]:
#!pip install tensorflow-gpu==2.0.0.alpha0

In [2]:
!pip install tqdm

### Downloading the Dogs vs Cats dataset 

In [3]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O ./cats_and_dogs_filtered.zip

--2020-10-25 09:12:20--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.197.128, 74.125.195.128, 74.125.142.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.197.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘./cats_and_dogs_filtered.zip’

./cats_and_dogs_fil 100%[===================>]  65.43M   115MB/s    in 0.6s    

2020-10-25 09:12:21 (115 MB/s) - ‘./cats_and_dogs_filtered.zip’ saved [68606236/68606236]



## Stage 2: Dataset preprocessing

### Import project dependencies

In [4]:
import os
import zipfile
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook
from tensorflow.keras.preprocessing.image import ImageDataGenerator

%matplotlib inline
tf.__version__

'2.3.0'

### Unzipping the Dogs vs Cats dataset

In [5]:
dataset_path = "./cats_and_dogs_filtered.zip"

In [6]:
zip_object = zipfile.ZipFile(file=dataset_path, mode="r")

In [7]:
zip_object.extractall("./")

In [8]:
zip_object.close()

### Seting up dataset paths

In [9]:
dataset_path_new = "./cats_and_dogs_filtered/"

In [10]:
train_dir = os.path.join(dataset_path_new, "train")
validation_dir = os.path.join(dataset_path_new, "validation")

## Building the model

### Loading the pre-trained model (MobileNetV2)

In [11]:
IMG_SHAPE = (128, 128, 3)

In [12]:
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE, include_top=False, weights="imagenet")

9412608/9406464 [==============================] - 0s 0us/step


In [13]:
# base_model.summary()

### Freezing the base model

In [14]:
base_model.trainable = False

### Defining the custom head for our network

In [15]:
base_model.output

<tf.Tensor 'out_relu/Relu6:0' shape=(None, 4, 4, 1280) dtype=float32>

In [16]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)

In [17]:
global_average_layer

<tf.Tensor 'global_average_pooling2d/Mean:0' shape=(None, 1280) dtype=float32>

In [18]:
prediction_layer = tf.keras.layers.Dense(units=1, activation='sigmoid')(global_average_layer)

### Defining the model

In [19]:
model = tf.keras.models.Model(inputs=base_model.input, outputs=prediction_layer)

In [20]:
# model.summary()

### Compiling the model

In [21]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

### Creating Data Generators

Resizing images

    Big pre-trained architecture support only certain input sizes.

For example: MobileNet (architecture that we use) supports: (96, 96), (128, 128), (160, 160), (192, 192), (224, 224).

In [22]:
data_gen_train = ImageDataGenerator(rescale=1/255.)
data_gen_valid = ImageDataGenerator(rescale=1/255.)

In [23]:
train_generator = data_gen_train.flow_from_directory(train_dir, target_size=(128,128), batch_size=128, class_mode="binary")

Found 2000 images belonging to 2 classes.


In [24]:
valid_generator = data_gen_valid.flow_from_directory(validation_dir, target_size=(128,128), batch_size=128, class_mode="binary")

Found 1000 images belonging to 2 classes.


### Training the model

In [25]:
model.fit_generator(train_generator, epochs=5, validation_data=valid_generator)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/5
16/16 [==============================] - 9s 580ms/step - loss: 0.7117 - accuracy: 0.5610 - val_loss: 0.6348 - val_accuracy: 0.6440
Epoch 2/5
16/16 [==============================] - 8s 508ms/step - loss: 0.6030 - accuracy: 0.6595 - val_loss: 0.5503 - val_accuracy: 0.7230
Epoch 3/5
16/16 [==============================] - 8s 505ms/step - loss: 0.5249 - accuracy: 0.7375 - val_loss: 0.4805 - val_accuracy: 0.7830
Epoch 4/5
16/16 [==============================] - 8s 504ms/step - loss: 0.4605 - accuracy: 0.7975 - val_loss: 0.4218 - val_accuracy: 0.8310
Epoch 5/5
16/16 [==============================] - 8s 505ms/step - loss: 0.4056 - accuracy: 0.8310 - val_loss: 0.3723 - val_accuracy: 0.8610


### Transfer learning model evaluation

In [26]:
valid_loss, valid_accuracy = model.evaluate_generator(valid_generator)

Instructions for updating:
Please use Model.evaluate, which supports generators.


In [27]:
print("Accuracy after transfer learning: {}".format(valid_accuracy))

Accuracy after transfer learning: 0.8610000014305115


## Fine tuning


There are a few pointers:

- DO NOT use Fine tuning on the whole network; only a few top layers are enough. In most cases, they are more specialized. The goal of the Fine-tuning is to adopt that specific part of the network for our custom (new) dataset.
- Start with the fine tunning AFTER you have finished with transfer learning step. If we try to perform Fine tuning immediately, gradients will be much different between our custom head layer and a few unfrozen layers from the base model. 

### Un-freeze a few top layers from the model

In [28]:
base_model.trainable = True

In [29]:
print("Number of layersin the base model: {}".format(len(base_model.layers)))

Number of layersin the base model: 155


In [30]:
fine_tune_at = 100

In [31]:
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

### Compiling the model for fine-tuning

In [32]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

### Fine tuning

In [33]:
model.fit_generator(train_generator,  
                    epochs=5, 
                    validation_data=valid_generator)

Epoch 1/5
16/16 [==============================] - 9s 557ms/step - loss: 0.1892 - accuracy: 0.9225 - val_loss: 0.0874 - val_accuracy: 0.9630
Epoch 2/5
16/16 [==============================] - 8s 511ms/step - loss: 0.0254 - accuracy: 0.9970 - val_loss: 0.0939 - val_accuracy: 0.9630
Epoch 3/5
16/16 [==============================] - 8s 513ms/step - loss: 0.0136 - accuracy: 0.9985 - val_loss: 0.0880 - val_accuracy: 0.9680
Epoch 4/5
16/16 [==============================] - 8s 512ms/step - loss: 0.0056 - accuracy: 1.0000 - val_loss: 0.1001 - val_accuracy: 0.9660
Epoch 5/5
16/16 [==============================] - 8s 511ms/step - loss: 0.0017 - accuracy: 1.0000 - val_loss: 0.1379 - val_accuracy: 0.9620


### Evaluating the fine tuned model

In [34]:
valid_loss, valid_accuracy = model.evaluate_generator(valid_generator)

In [35]:
print("Validation accuracy after fine tuning: {}".format(valid_accuracy))

Validation accuracy after fine tuning: 0.9620000123977661
